In [1]:
import sys
sys.path.append('/home/phzd/AI/bidd-molmap_v1_3')
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import auc as calculate_auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os,time,json
from molmap import feature,loadmap
from scipy.stats.stats import pearsonr
from argparse import ArgumentParser
import xgboost

In [2]:
def PRC_AUC(y_true, y_score):
	precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
	auc = calculate_auc(recall, precision)
	return auc

def ROC_AUC(y_true, y_score):
	auc = roc_auc_score(y_true, y_score)
	return auc

# Load y

In [3]:
df=pd.read_csv('CB2_Chembl_C7758.csv')

In [4]:
df.head()

,smiles,act1,act2,act3,act4
0,Cc1cccc(-c2nnc(SCC(=O)NC3CCCCC3)n2Cc2ccccc2)c1,1,1,1,1
1,CCCCSc1nc2c(c(=O)n1CCc1ccccc1)C1(CCCC1)Cc1cccc...,1,1,1,1
2,CCOc1ccc(N(CC(=O)N[C@@H](C)CC)S(=O)(=O)c2ccc(B...,1,1,1,1
3,CCCCCN(CCCCC)C(=O)c1ccccc1C(=O)Nc1ccc(Br)cc1,1,1,1,1
4,CCCCCOc1c(OC)ccc2cc(C(=O)NCCc3ccncc3)c(=O)[nH]c12,1,1,1,1


In [5]:
Y0 = df['threshold4'].astype('int').values

In [6]:
Y0

array([1, 1, 1, ..., 0, 0, 0])

In [7]:
len(Y0)

7758

# Load feats, i.e. X

In [9]:
X0=pd.read_csv('AvalonFP_MorganFP_AtomPairFP.csv')

In [10]:
X0.head()

,AvalonFP0,AvalonFP1,AvalonFP2,AvalonFP3,AvalonFP4,AvalonFP5,AvalonFP6,AvalonFP7,AvalonFP8,AvalonFP9,...,AtomPairFP2038,AtomPairFP2039,AtomPairFP2040,AtomPairFP2041,AtomPairFP2042,AtomPairFP2043,AtomPairFP2044,AtomPairFP2045,AtomPairFP2046,AtomPairFP2047
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,1,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
X0.iloc[:2,2046:2049]

,AvalonFP2046,AvalonFP2047,MorganFP0
0,1,1,0
1,0,0,0


In [12]:
X0.iloc[:2,4094:4097]

,MorganFP2046,MorganFP2047,AtomPairFP0
0,0,0,0
1,0,0,0


In [13]:
len(X0)

7758

# splitting data in random mode, using seed=1
# i.e. X0,Y0 will be splitted into  train, valid, test dataset

In [14]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [15]:
seed = 1
X0=X0.values

In [16]:
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

6208 775 775
6208 775 775


# loading optimized parameters

In [17]:
import json
with open('params_class','r') as f:
    best_param=json.load(f)

In [18]:
best_param

{'params': {'base_score': 0.5,
  'booster': 'gbtree',
  'colsample_bylevel': 1,
  'colsample_bynode': 1,
  'colsample_bytree': 0.66,
  'gamma': 0,
  'gpu_id': 0,
  'learning_rate': 0.05,
  'max_delta_step': 1,
  'max_depth': 8,
  'min_child_weight': 5,
  'missing': None,
  'n_estimators': 1500,
  'n_jobs': 1,
  'nthread': None,
  'objective': 'binary:logistic',
  'random_state': 0,
  'reg_alpha': 0,
  'reg_lambda': 1,
  'scale_pos_weight': 1,
  'seed': 123,
  'silent': None,
  'subsample': 0.53,
  'tree_method': 'gpu_hist',
  'verbosity': 1},
 'results': {'fp_types': 'AtomPairFP_MorganFP_AvalonFP',
  'model': 'xgb',
  'seed': 1,
  'test_roc_auc': 0.9295218351557525,
  'test_roc_aucs': '[0.9295218351557525]',
  'time_fit': 25.68925952911377,
  'valid_roc_auc': 0.9343158953722335,
  'valid_roc_aucs': '[0.9343158953722335]',
  'values_col': 'act4'}}

In [19]:
use_param=best_param['params']

In [20]:
use_param

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.66,
 'gamma': 0,
 'gpu_id': 0,
 'learning_rate': 0.05,
 'max_delta_step': 1,
 'max_depth': 8,
 'min_child_weight': 5,
 'missing': None,
 'n_estimators': 1500,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 123,
 'silent': None,
 'subsample': 0.53,
 'tree_method': 'gpu_hist',
 'verbosity': 1}

# define xgboost classification model

In [21]:
clf=xgboost.XGBClassifier(**use_param)

In [22]:
print('clf = ',clf)

clf =  XGBClassifier(colsample_bytree=0.66, gpu_id=0, learning_rate=0.05,
              max_delta_step=1, max_depth=8, min_child_weight=5,
              n_estimators=1500, seed=123, subsample=0.53,
              tree_method='gpu_hist')


# training xgb using training dataset

In [23]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

fit time is: 55.349159717559814


# evaluating the performance by valid and test dataset

In [25]:
clf.predict_proba(X_test)

array([[2.5838614e-04, 9.9974161e-01],
       [1.9616485e-03, 9.9803835e-01],
       [3.8714582e-01, 6.1285418e-01],
       ...,
       [3.5385489e-03, 9.9646145e-01],
       [1.8198323e-01, 8.1801677e-01],
       [1.5120411e-01, 8.4879589e-01]], dtype=float32)

In [26]:
clf.predict_proba(X_valid)

array([[0.98645663, 0.01354336],
       [0.07401276, 0.92598724],
       [0.99448264, 0.00551737],
       ...,
       [0.78553116, 0.21446887],
       [0.94328004, 0.05671994],
       [0.01339525, 0.98660475]], dtype=float32)

In [28]:
clf.predict_proba(X_test)[:3,1]

array([0.9997416 , 0.99803835, 0.6128542 ], dtype=float32)

In [31]:
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

valid_roc_auc, test_roc_auc =  0.9335311871227363 0.9287698332046121


In [34]:
results = {"seed":seed, 'valid_roc_auc':valid_roc_auc,"test_roc_auc":test_roc_auc, "time":time_fit}
print('results = ',results)

results =  {'seed': 1, 'valid_roc_auc': 0.9335311871227363, 'test_roc_auc': 0.9287698332046121, 'time': 55.349159717559814}


# change random seed to 8 to split the X0 and Y0, rerun the training and evaluation

In [35]:
seed = 8
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

6208 775 775
6208 775 775


# re-define xgb model

In [37]:
clf=''
clf=xgboost.XGBClassifier(**use_param)

In [38]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)
results = {"seed":seed, 'valid_roc_auc':valid_roc_auc,"test_roc_auc":test_roc_auc, "time":time_fit}
print('results = ',results)

fit time is: 55.828707456588745
valid_roc_auc, test_roc_auc =  0.9356163645667808 0.9373565763517051
results =  {'seed': 8, 'valid_roc_auc': 0.9356163645667808, 'test_roc_auc': 0.9373565763517051, 'time': 55.828707456588745}
